In [7]:

import pandas as pd
import numpy as np
import gpxpy
import gpxpy.gpx
import matplotlib.pyplot as plt
import folium
from IPython.display import display
import plotly.express as px
import haversine as hs
import plotly.graph_objects as go
import tqdm
import math
import requests



In [8]:
def haversine_distance(lat1, lon1, lat2, lon2) -> float:
    distance = hs.haversine(
        point1=(lat1, lon1),
        point2=(lat2, lon2),
        unit=hs.Unit.METERS
    )
    return np.round(distance, 2)

def attribute_calc(gpx):
    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation
                })
    route_df = pd.DataFrame(route_info)
    
    
    haversine_distance(
        lat1=route_df.iloc[0]['latitude'],
        lon1=route_df.iloc[0]['longitude'],
        lat2=route_df.iloc[1]['latitude'],
        lon2=route_df.iloc[1]['longitude']
    )    
        
    distances = [np.nan]

    for i in range(len(route_df)):
        if i == 0:
            continue
        else:
            distances.append(haversine_distance(
                lat1=route_df.iloc[i - 1]['latitude'],
                lon1=route_df.iloc[i - 1]['longitude'],
                lat2=route_df.iloc[i]['latitude'],
                lon2=route_df.iloc[i]['longitude']
            ))
            
    route_df['distance'] = distances
    
    route_df['elevation_diff'] = route_df['elevation'].diff()

    route_df[route_df['elevation_diff'] >= 0]['elevation_diff'].sum()
    route_df['distance'].sum()
    route_df['cum_elevation'] = route_df['elevation_diff'].cumsum()
    route_df['cum_distance'] = route_df['distance'].cumsum()
    route_df = route_df.fillna(0)
    
    return(route_df)

def calcBearing (lat1, long1, lat2, long2):
    dLon = (long2 - long1)
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
    bearing = math.atan2(x,y)   # use atan2 to determine the quadrant
    bearing = math.degrees(bearing)

    return bearing

def calcNSEW(lat1, long1, lat2, long2):
    points = ["north", "north east", "east", "south east", "south", "south west", "west", "north west"]
    bearing = calcBearing(lat1, long1, lat2, long2)
    bearing += 22.5
    bearing = bearing % 360
    bearing = int(bearing / 45) # values 0 to 7
    NSEW = points [bearing]

    return NSEW

def infogatherer(gpx):
    
    for track in gpx.tracks:
        TrackName = track.name
        #print(gpx.metadata.author)
        TrackDesc = track.description
    RouteDetails = attribute_calc(gpx)
    Info = {}
    Info["Date"] = track.name
    Info["Start"] = track.description
    LengthDraft = RouteDetails.distance.sum()/1000
    Info["Length"] = LengthDraft.round(2)
    Info["Elevation Gain"] = int(RouteDetails[RouteDetails['elevation_diff'] > 0]['elevation_diff'].sum().round())
    lat_start = RouteDetails.iloc[0][0]
    lon_start = RouteDetails.iloc[0][1]
    lat_center = RouteDetails.latitude.median()
    lon_center = RouteDetails.longitude.median()
    Info["Heading"] = calcNSEW( lat_start, lon_start, lat_center, lon_center)
    
    return(Info)



In [9]:
#print (requests.head('https://raw.githubusercontent.com/MannuelTe/weekdayDB/files/010.06.2021.gpx').status_code)
viable_url = []
for y in [20,21,22]:
    for m in range(0,13):
        nice_m = "{:02d}".format(m)
        for d in range(0,31):
            nice_d = "{:02d}".format(d)
            url_pot = fr'https://raw.githubusercontent.com/MannuelTe/weekdayDB/files/{nice_d}.{nice_m}.20{y}.gpx'
            if requests.head(url_pot).status_code == 200:
                print(fr"{nice_d}.{nice_m}.20{y}")
                viable_url.append(url_pot)

In [ ]:
len(viable_url)
    

45

In [ ]:
daframe_draft = []
for url in tqdm.tqdm(viable_url):
    download = requests.get(url).content.decode("UTF-8")

    #parse gpx
    gpx  = gpxpy.parse(download)
    daframe_draft.append(infogatherer(gpx))
RoutesDF = pd.DataFrame.from_dict(daframe_draft)
RoutesDF["Date"] = pd.to_datetime(RoutesDF["Date"], dayfirst=True)
RoutesDF.head()

100%|██████████| 45/45 [00:24<00:00,  1.82it/s]


,Date,Start,Length,Elevation Gain,Heading
0,2020-05-10,Frohburg,76.71,1367,south east
1,2020-06-09,Fork,72.29,1012,south
2,2020-06-11,Frohburg,74.85,1109,south east
3,2020-06-16,Fork,76.18,1328,south
4,2020-06-30,Frohburg,78.18,1362,south east


In [ ]:
csv_path = r"C:\Users\trach\Desktop\Route_DB\routes_DB.csv"
RoutesDF.to_csv(csv_path, index = False)

**UPLOAD THE NEW CSV TO GITHUB**